In [396]:
import math ; pi=math.pi
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib widget
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
import time
import json
import imp
np.warnings.filterwarnings('ignore')
import PySSC
import PySSC_AP

In [397]:
def plot_diag(Sij1,Sij2,name1,name2,ylims):
    fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, gridspec_kw=dict(height_ratios=[2,1]),figsize=(10,5))
    nbins = len(np.diag(Sij1))
    ax1.semilogy(np.arange(nbins),np.diag(Sij1),label='%s'%name1)
    ax1.plot(np.arange(nbins),np.diag(Sij2),label='%s'%name2)
    ax2.set_xlabel('bins')
    ax1.set_ylabel('$S_{ii}$')
    ax2.set_ylabel('dev in $[\%]$')
    ax1.legend()
    ax1.grid(alpha=0.2); ax2.grid(alpha=0.2)
    ax2.set_ylim(ylims)
    ax2.plot(np.arange(nbins),100*(np.diag(Sij1)/np.diag(Sij2)-1),'.')
    ax2.plot(np.arange(nbins),np.zeros(nbins),'r--',color='black',linewidth=0.7)
    fig.subplots_adjust(hspace=.0)
    fig.show()

def plotting_offD(Sij):
    cov_vector = []
    for j,i in itertools.product(range(Sij.shape[0]),range(Sij.shape[1])):
        if i>=j:
            cov_vector.append(Sij[i,j])
    return (np.array(cov_vector));

def plot_offdiag(Sij1,Sij2,name1,name2,ylims):
    cov_vector1 = plotting_offD(Sij1)
    cov_vector2 = plotting_offD(Sij2)
    diff = ((Sij1 / Sij2)-1)*100
    cov_vector3 = plotting_offD(diff)
    fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, gridspec_kw=dict(height_ratios=[2,1]),figsize=(10,5))
    ax1.set_ylabel('$S_{ij}$',fontsize=12);ax2.set_ylabel('dev in $[\%]$',fontsize=11.5)
    ax2.set_xlabel('matrix index',fontsize=12)
    ax1.semilogy(np.arange(len(cov_vector1)),abs(cov_vector1),linewidth='1',label='%s'%name1)
    ax1.semilogy(np.arange(len(cov_vector2)),abs(cov_vector2),'r--',linewidth='0.7',label='%s'%name2)
    ax2.plot(np.arange(len(cov_vector3)),cov_vector3,'.')
    ax2.plot(np.arange(len(cov_vector3)),np.zeros(len(cov_vector3)),'r--',color='black',linewidth=0.7)
    ax2.set_ylim(ylims)
    ax2.fill_between(np.arange(len(cov_vector3)),5,-5,color='silver',alpha=0.5)
    ax1.grid(alpha=0.2);    ax2.grid(alpha=0.2)
    ax1.legend(fontsize=13,loc=(0.72,0.05))
    fig.subplots_adjust(hspace=.0)
    fig.show()

In [398]:
# If you want to change cosmology, specify the parameters with a dictionnary in the format of CLASS :
params = {'omega_b':0.022,'omega_cdm':0.12,'H0':67.,'n_s':0.96,'sigma8':0.81}

In [399]:
zstakes = np.linspace(0.2,1.5,num=14)
zmin = np.min(zstakes) ; zmax = np.max(zstakes)
# we have zmin zmax and 14 intermediates redshifts that define some boundaries

In [490]:
# Define redshift range
nz       = 700
z_arr    = np.linspace(0,2,num=nz+1)[1:] # Redshifts must be > 0

# TopHat window

In [491]:
nbins_T   = len(zstakes)-1
windows_T = np.zeros((nbins_T,nz))
for i in range(nbins_T):
    zminbin = zstakes[i] ; zmaxbin = zstakes[i+1] ; Dz = zmaxbin-zminbin
    for iz in range(nz):
        z = z_arr[iz]
        if ((z>zminbin) and (z<=zmaxbin)):
            windows_T[i,iz] = 1/Dz

In [492]:
imp.reload(PySSC_AP) #if needed

start_time = time.time()
Sijw_T = PySSC.Sij(z_arr,windows_T)
print(time.time() - start_time)
start_time = time.time()
Sijw_T_AngPow = PySSC_AP.Sij_AngPow_psky(z_arr,windows_T)
print(time.time() - start_time)

3.616166353225708
59.86253118515015


In [493]:
plot_diag(Sijw_T,Sijw_T_AngPow,'Sijw_T','Sijw_T_AngPow',(-10,10))
plot_offdiag(Sijw_T,Sijw_T_AngPow,'Sijw_T','Sijw_T_AngPow',(-50,50))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gaussian window 1

In [494]:
sigmaz    = 0.01
zcenter_G = [0.4,0.55,0.7,0.85,1.,1.15,1.3,1.45,1.6]
nbins_G   = len(zcenter_G)
windows_G = np.zeros((nbins_G,nz))
for i in range(nbins_G):
    windows_G[i,:] = np.exp(-(z_arr-zcenter_G[i])**2/(2*sigmaz**2)) / np.sqrt(2*pi*sigmaz**2)

# Plot window functions
fig,ax=plt.subplots(1,1)
for i in range(nbins_G):
    ax.plot(z_arr,windows_G[i,:])
ax.set_xlabel('z') ; ax.set_ylabel('$W_i(z)$')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [495]:
imp.reload(PySSC_AP) #if needed
imp.reload(PySSC)

start_time = time.time()
Sijw_G = PySSC.Sij(z_arr,windows_G)
print(time.time() - start_time)
start_time = time.time()
Sijw_G_AngPow = PySSC_AP.Sij_AngPow_psky(z_arr,windows_G)
print(time.time() - start_time)

3.12756609916687
33.819353103637695


In [496]:
plot_diag   (Sijw_G,Sijw_G_AngPow,'Sijw_G','Sijw_G_AngPow',(-10,10))
plot_offdiag(Sijw_G,Sijw_G_AngPow,'Sijw_G','Sijw_G_AngPow',(-100,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gaussian window 2

In [497]:
sigmaz    = 0.04
zcenter_G = [0.4,0.55,0.7,0.85,1.,1.15,1.3,1.45,1.6]
nbins_G   = len(zcenter_G)
windows_G = np.zeros((nbins_G,nz))
for i in range(nbins_G):
    windows_G[i,:] = np.exp(-(z_arr-zcenter_G[i])**2/(2*sigmaz**2)) / np.sqrt(2*pi*sigmaz**2)

# Plot window functions
fig,ax=plt.subplots(1,1)
for i in range(nbins_G):
    ax.plot(z_arr,windows_G[i,:])
ax.set_xlabel('z') ; ax.set_ylabel('$W_i(z)$')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [498]:
imp.reload(PySSC_AP) #if needed
imp.reload(PySSC)

start_time = time.time()
Sijw_G2 = PySSC.Sij(z_arr,windows_G)
print(time.time() - start_time)
start_time = time.time()
Sijw_G_AngPow2 = PySSC_AP.Sij_AngPow_psky(z_arr,windows_G)
print(time.time() - start_time)

2.9394166469573975
30.949292182922363


In [499]:
plot_diag   (Sijw_G2,Sijw_G_AngPow2,'Sijw_G2','Sijw_G_AngPow2',(-20,20))
plot_offdiag(Sijw_G2,Sijw_G_AngPow2,'Sijw_G2','Sijw_G_AngPow2',(-20,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Partial sky, same examples as "Partsky_examples.ipynb"

In [427]:
imp.reload(PySSC_AP) #if needed
imp.reload(PySSC)

<module 'PySSC' from '/renoir/baratta/software/PySSC/PySSC.py'>

In [428]:
# Redshift bins
zstakes = np.linspace(0.2,1.5,num=14)
zmin = np.min(zstakes) ; zmax = np.max(zstakes)
# Window function
nz       = 500
z_arr    = np.linspace(0,2,num=nz+1)[1:] # Redshifts must be > 0
nbins_T   = len(zstakes)-1
windows_T = np.zeros((nbins_T,nz))
for i in range(nbins_T):
    zminbin = zstakes[i] ; zmaxbin = zstakes[i+1] ; Dz = zmaxbin-zminbin
    for iz in range(nz):
        z = z_arr[iz]
        if ((z>zminbin) and (z<=zmaxbin)):
            windows_T[i,iz] = 1/Dz

## Comparison of Sij matrix for full sky and partial sky Sij for full-sky mask

In [429]:
# full sky Sij
t0 = time.clock()
print('Full sky Sij')
Sijw_full = PySSC.Sij(z_arr,windows_T)
# partial sky Sij with full-sky mask
t1 = time.clock()
print('Partial sky Sij')
Sijw_part = PySSC.Sij_psky(z_arr,windows_T,clmask=None,mask='./masks/full_sky_map.fits') #long computation
t2 = time.clock()
print('AngPow Sij')
Sijw_AP = PySSC_AP.Sij_AngPow(z_arr,windows_T,clmask=None,mask='./masks/full_sky_map.fits',verbose=True)
t3 = time.clock()
print('Sij full sky took: %.1f secs, part sky %.1f secs, AngPow %.1f secs' %(t1-t0,t2-t1,t3-t2))

Full sky Sij
Partial sky Sij
0.5592684397700074
AngPow Sij
Using mask map, given as a fits file
f_sky = 1.0000
lmax = 1
Sij full sky took: 17.9 secs, part sky 93.2 secs, AngPow 95.3 secs


In [19]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=3,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_full)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_part)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im3 = axes[2].imshow(np.log(abs(Sijw_AP)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Full sky Sij')
axes[1].set_title('Partial sky Sij with full sky mask')
axes[2].set_title('AngPow Sij with full sky mask')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plot_diag   (Sijw_part,Sijw_AP,'Sijw_part','Sijw_AP',(-10,3))
plot_offdiag(Sijw_part,Sijw_AP,'Sijw_part','Sijw_AP',(-30,30))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for DES mask

In [21]:
t0 = time.clock()
Sijw_part_DES = PySSC.Sij_psky(z_arr,windows_T,mask='./masks/DES-mask-simple-ring-1024.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_DES = PySSC_AP.Sij_AngPow(z_arr,windows_T,mask='./masks/DES-mask-simple-ring-1024.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.1207
lmax = 32
0.5592684397700074
Computed in 7.6 minutes
Using mask map, given as a fits file
f_sky = 0.1207
lmax = 32
Computed in 1.6 minutes


In [22]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_DES)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_DES)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for DES mask , partsky')
axes[1].set_title('Sij for DES mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plot_diag   (Sijw_part_DES,Sijw_AngPow_DES,'Sijw_part_DES','Sijw_AngPow_DES',(-10,3))
plot_offdiag(Sijw_part_DES,Sijw_AngPow_DES,'Sijw_part_DES','Sijw_AngPow_DES',(-30,30))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for Euclid mask

In [24]:
t0 = time.clock()
Sijw_part_Euclid = PySSC.Sij_psky(z_arr,windows_T,mask='./masks/Euclid_map_WIDE_SURVEY.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_Euclid = PySSC_AP.Sij_AngPow(z_arr,windows_T,mask='./masks/Euclid_map_WIDE_SURVEY.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.3762
lmax = 27
0.5592684397700074
Computed in 5.5 minutes
Using mask map, given as a fits file
f_sky = 0.3762
lmax = 27
Computed in 0.4 minutes


In [25]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_Euclid)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_Euclid)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for Euclid mask , partsky')
axes[1].set_title('Sij for Euclid mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plot_diag   (Sijw_part_Euclid,Sijw_AngPow_Euclid,'Sijw_part_Euclid','Sijw_AngPow_Euclid',(-10,3))
plot_offdiag(Sijw_part_Euclid,Sijw_AngPow_Euclid,'Sijw_part_Euclid','Sijw_AngPow_Euclid',(-30,30))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for 5 degrees patch

In [27]:
t0 = time.clock()
Sijw_part_5d = PySSC.Sij_psky(z_arr,windows_T,mask='./masks/circular-mask_5deg.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_5d = PySSC_AP.Sij_AngPow(z_arr,windows_T,mask='./masks/circular-mask_5deg.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.0019
lmax = 136
0.5592684397700074
Computed in 26.7 minutes
Using mask map, given as a fits file
f_sky = 0.0019
lmax = 136
Computed in 1.6 minutes


In [28]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_5d)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_5d)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for Euclid mask , partsky')
axes[1].set_title('Sij for Euclid mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
plot_diag   (Sijw_part_5d,Sijw_AngPow_5d,'Sijw_part_5d','Sijw_AngPow_5d',(-10,3))
plot_offdiag(Sijw_part_5d,Sijw_AngPow_5d,'Sijw_part_5d','Sijw_AngPow_5d',(-100,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Same but with gaussian window

In [30]:
imp.reload(PySSC_AP)
imp.reload(PySSC)

<module 'PySSC' from '/renoir/baratta/software/PySSC/PySSC.py'>

In [31]:
#Top-hat window functions have no interest here, since the cross-spectra would be basically zero
#So we go for Gaussian window functions
sigmaz    = 0.05
zcenter_G = [0.4,0.55,0.7,0.85,1.,1.15,1.3,1.45,1.6]
nbins_G   = len(zcenter_G)
windows_G = np.zeros((nbins_G,nz))
for i in range(nbins_G):
    windows_G[i,:] = np.exp(-(z_arr-zcenter_G[i])**2/(2*sigmaz**2)) / np.sqrt(2*pi*sigmaz**2)

# Plot window functions
for i in range(nbins_G):
    plt.plot(z_arr,windows_G[i,:])
plt.xlabel('z') ; plt.ylabel('$W_i(z)$')
plt.show()

In [32]:
imp.reload(PySSC_AP)
imp.reload(PySSC)
# full sky Sij
t0 = time.clock()
print('Full sky Sij')
Sijw_fullG = PySSC.Sij(z_arr,windows_G)
# partial sky Sij with full-sky mask
t1 = time.clock()
print('Partial sky Sij')
Sijw_partG = PySSC.Sij_psky(z_arr,windows_G,clmask=None,mask='./masks/full_sky_map.fits') #long computation
t2 = time.clock()
print('AngPow Sij')
Sijw_APG = PySSC_AP.Sij_AngPow(z_arr,windows_G,clmask=None,mask='./masks/full_sky_map.fits',verbose=True)
t3 = time.clock()
print('Sij full sky took: %.1f secs, part sky %.1f secs, AngPow %.1f secs' %(t1-t0,t2-t1,t3-t2))

Full sky Sij
Partial sky Sij
0.5592684397700074
AngPow Sij
Using mask map, given as a fits file
f_sky = 1.0000
lmax = 1
Sij full sky took: 18.7 secs, part sky 96.5 secs, AngPow 92.2 secs


In [33]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=3,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_fullG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_partG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im3 = axes[2].imshow(np.log(abs(Sijw_APG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Full sky Sij')
axes[1].set_title('Partial sky Sij with full sky mask')
axes[2].set_title('AngPow Sij with full sky mask')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plot_diag   (Sijw_partG,Sijw_APG,'Sijw_partG','Sijw_APG',(-10,10))
plot_offdiag(Sijw_partG,Sijw_APG,'Sijw_partG','Sijw_APG',(-10,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for DES mask

In [35]:
imp.reload(PySSC_AP)
t0 = time.clock()
Sijw_part_DESG = PySSC.Sij_psky(z_arr,windows_G,mask='./masks/DES-mask-simple-ring-1024.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_DESG = PySSC_AP.Sij_AngPow(z_arr,windows_G,mask='./masks/DES-mask-simple-ring-1024.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.1207
lmax = 32
0.5592684397700074
Computed in 6.0 minutes
Using mask map, given as a fits file
f_sky = 0.1207
lmax = 32
Computed in 1.6 minutes


In [36]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_DESG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_DESG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for DES mask , partsky')
axes[1].set_title('Sij for DES mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plot_diag   (Sijw_part_DESG,Sijw_AngPow_DESG,'Sijw_part_DESG','Sijw_AngPow_DESG',(-10,10))
plot_offdiag(Sijw_part_DESG,Sijw_AngPow_DESG,'Sijw_part_DESG','Sijw_AngPow_DESG',(-10,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for Euclid mask

In [38]:
t0 = time.clock()
Sijw_part_EuclidG = PySSC.Sij_psky(z_arr,windows_G,mask='./masks/Euclid_map_WIDE_SURVEY.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_EuclidG = PySSC_AP.Sij_AngPow(z_arr,windows_G,mask='./masks/Euclid_map_WIDE_SURVEY.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.3762
lmax = 27
0.5592684397700074
Computed in 4.3 minutes
Using mask map, given as a fits file
f_sky = 0.3762
lmax = 27
Computed in 0.4 minutes


In [39]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_EuclidG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_EuclidG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for Euclid mask , partsky')
axes[1].set_title('Sij for Euclid mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
plot_diag   (Sijw_part_EuclidG,Sijw_AngPow_EuclidG,'Sijw_part_EuclidG','Sijw_AngPow_EuclidG',(-10,10))
plot_offdiag(Sijw_part_EuclidG,Sijw_AngPow_EuclidG,'Sijw_part_EuclidG','Sijw_AngPow_EuclidG',(-10,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Partial sky Sij for 5 degrees patch

In [41]:
t0 = time.clock()
Sijw_part_5dG = PySSC.Sij_psky(z_arr,windows_G,mask='./masks/circular-mask_5deg.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))
t0 = time.clock()
Sijw_AngPow_5dG = PySSC_AP.Sij_AngPow(z_arr,windows_G,mask='./masks/circular-mask_5deg.fits',verbose=True)
t1 = time.clock()
print('Computed in %.1f minutes' %((t1-t0)/60))

Using mask map, given as a fits file
f_sky = 0.0019
lmax = 136
0.5592684397700074
Computed in 21.1 minutes
Using mask map, given as a fits file
f_sky = 0.0019
lmax = 136
Computed in 1.5 minutes


In [42]:
# Plotting the two matrices
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=(8,3.5))
im = axes[0].imshow(np.log(abs(Sijw_part_5dG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
im2 = axes[1].imshow(np.log(abs(Sijw_AngPow_5dG)),interpolation='none',cmap='bwr',extent=[zmin,zmax,zmax,zmin],vmin=-20,vmax=-10)
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
axes[0].set_title('Sij for Euclid mask , partsky')
axes[1].set_title('Sij for Euclid mask , AngPow')
fig.subplots_adjust(left=0.02,bottom=0.08,top=0.9,wspace=0.002)
cbar_ax = fig.add_axes([0.9, 0.08, 0.02, 0.87])
c_bar=fig.colorbar(im,cax=cbar_ax,fraction=.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
plot_diag   (Sijw_part_5dG,Sijw_AngPow_5dG,'Sijw_part_5dG','Sijw_AngPow_5dG',(-10,10))
plot_offdiag(Sijw_part_5dG,Sijw_AngPow_5dG,'Sijw_part_5dG','Sijw_AngPow_5dG',(-10,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Euclid redshift bins

In [44]:
prefix_Euclid='./Euclid_files/'

## Photometric bins

In [45]:
zarr = np.loadtxt(prefix_Euclid+'n_z_without_1_plus_z.dat',unpack=True,usecols=(0))[100:]
window = np.loadtxt(prefix_Euclid+'n_z_without_1_plus_z.dat',unpack=True,usecols=(1,2,3,4,5,6,7,8,9,10))[:,100:]
nbins = window.shape[0]
zbins = zarr[np.argmax(window,axis=1)]
plt.figure()
for u,zbin in enumerate(window):
	plt.plot(zarr,zbin,linewidth=2)
plt.xlabel(r'$z$',fontsize=20) ; plt.xticks(fontsize=15)
plt.ylabel(r'$n(z)$',fontsize=20)
# plt.legend(frameon='False',loc='best')
plt.xlim([-0.1,3])
plt.tight_layout()
plt.savefig(prefix_Euclid+'n_z_Euclid_GCphot.png')
plt.show()
print(zbins)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.35   0.5065 0.628  0.737  0.843  0.9525 1.0735 1.217  1.408  1.691 ]


In [46]:
imp.reload(PySSC_AP)
imp.reload(PySSC)
# partial sky Sijkl with Euclid mask
t1 = time.clock()
print('Computing partial sky Sijkl for Euclid with PySSC...')
Sij1_photo_PySSC = PySSC.Sij_psky(zarr,window,clmask=None,mask='./masks/Euclid_map_WIDE_SURVEY.fits') 
t2 = time.clock()
print('Sij took %.1f mins' %((t2-t1)/60))
t1 = time.clock()
print('Computing partial sky Sijkl for Euclid with AngPow...')
Sij1_photo_AngPow = PySSC_AP.Sij_AngPow(zarr,window,clmask=None,mask='./masks/Euclid_map_WIDE_SURVEY.fits') 
t2 = time.clock()
print('Sij took %.1f mins' %((t2-t1)/60))

Computing partial sky Sijkl for Euclid with PySSC...
0.29850746268656714
Sij took 15.6 mins
Computing partial sky Sijkl for Euclid with AngPow...
Sij took 0.5 mins


In [47]:
plot_diag   (Sij1_photo_PySSC,Sij1_photo_AngPow,'Sij1_photo_PySSC','Sij1_photo_AngPow',(-20,20))
plot_offdiag(Sij1_photo_PySSC,Sij1_photo_AngPow,'Sij1_photo_PySSC','Sij1_photo_AngPow',(-20,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Spectrometry

In [48]:
#GCs
zmins = np.array([0.90,1.10,1.30,1.50])
zmaxs = np.array([1.10,1.30,1.50,1.80])
n_zs = np.array([6.86,5.58,4.21,2.61])* 1e-4
zbins = np.array([1.00,1.20,1.40,1.65])
zmin = np.min(zmins) ; zmax = np.max(zmaxs)
zstakes    = np.r_[zmins,zmaxs[-1]]

In [49]:
# Define redshift range
nz       = 500
z_arr    = np.linspace(0.95*zmin,zmax*1.05,num=nz)

nbins2   = zmins.size
windows2 = np.zeros((nbins2,nz))
for i in range(nbins2):
    zminbin = zmins[i] ; zmaxbin = zmaxs[i] ; Dz = zmaxbin-zminbin
    for iz,z in enumerate(z_arr):
        if ((z>zminbin) and (z<=zmaxbin)):
            windows2[i,iz] = n_zs[i]

plt.figure()
for i in range(nbins2):   
    plt.plot(z_arr,windows2[i,:])
plt.xlabel(r'$z$',fontsize=20)
plt.ylabel(r'$n(z)$',fontsize=20)
plt.xticks(fontsize=15) ; plt.yticks(fontsize=8)
plt.tight_layout()
#plt.savefig(prefix_Euclid+'n_z_Euclid_GCs.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
imp.reload(PySSC_AP)
imp.reload(PySSC)
# partial sky Sijkl with Euclid mask
t1 = time.clock()
print('Computing partial sky Sijkl for Euclid with PySSC...')
Sij2_spectro_PySSC = PySSC.Sij_psky(z_arr,windows2,clmask=None,mask='./masks/Euclid_map_WIDE_SURVEY.fits')
t2 = time.clock()
print('Sij took %.1f mins' %((t2-t1)/60))

Computing partial sky Sijkl for Euclid with PySSC...
0.29850746268656714
Sij took 2.5 mins


In [51]:
imp.reload(PySSC_AP)
t1 = time.clock()
print('Computing partial sky Sijkl for Euclid with AngPow...')
Sij2_spectro_AngPow = PySSC_AP.Sij_AngPow(z_arr,windows2,clmask=None,mask='./masks/Euclid_map_WIDE_SURVEY.fits') 
t2 = time.clock()
print('Sij took %.1f mins' %((t2-t1)/60))

Computing partial sky Sijkl for Euclid with AngPow...
Sij took 0.4 mins


In [52]:
#plot_diag   (Sij2_spectro_PySSC,Sij2_spectro_AngPow,'Sij2_spectro_PySSC','Sij2_spectro_AngPow',(-20,20))
plot_offdiag(Sij2_spectro_PySSC,Sij2_spectro_AngPow,'Sij2_spectro_PySSC','Sij2_spectro_AngPow',(-20,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …